Asked ChatGPT how to convert bigrams into a network and it showed me the workflow which I corrected for errors and adapted to my data

In [1]:
from pyspark.sql import SparkSession
from graphframes import *
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from pyspark.sql import types as T

spark = SparkSession \
        .builder \
        .appName("network") \
        .getOrCreate()

spark_nlp = sparknlp.start()

24/11/19 19:12:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/11/19 19:12:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
data = spark.read.csv("../data_topicmodel.csv", header= True).select(["id", "cleaned_text"])
data = data.sample(0.001)

Preprocess and get tokens

In [3]:
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "even", "one", "bye", "take"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year", "now"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_"]

stopwords = english + time + reddit + topic_specific

In [4]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

In [5]:
tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

In [6]:
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized')

In [7]:
from sparknlp.base import Finisher

finisher = Finisher().setInputCols(['normalized'])

In [8]:
my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          finisher
      ])

In [9]:
pipelineModel = my_pipeline.fit(data)
processed_data = pipelineModel.transform(data)
processed_data.show(1)

+------+--------------------+--------------------+
|    id|        cleaned_text| finished_normalized|
+------+--------------------+--------------------+
|104xdd|i spend most of t...|[i, spend, most, ...|
+------+--------------------+--------------------+
only showing top 1 row



Create bigrams

In [10]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

def generate_bigrams(tokens):
    return [(tokens[i], tokens[i+1]) for i in range(len(tokens) - 1) \
           if tokens[i] not in stopwords and tokens[i+1] not in stopwords]
                    
bigram_udf = F.udf(generate_bigrams, T.ArrayType(T.ArrayType(T.StringType())))
bigrams_df = processed_data.withColumn("bigrams", bigram_udf(F.col("finished_normalized")))

bigrams_df.select('bigrams').show(10, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                             bigrams|
+----------------------------------------------------------------------------------------------------+
|[[frustratedly, bored], [whole, gone], [wind, turnipspeech], [watching, inuyasha], [found, vaguel...|
|[[pretty, long], [bold, lines], [huge, fan], [never, shied], [shied, away], [always, tried], [fit...|
|   [[never, felt], [felt, comfortable], [school, fixing], [needed, emotional], [emotional, support]]|
|[[never, ending], [ending, loop], [actually, becoming], [depression, holds], [nursing, school], [...|
|[[biggest, dream], [quite, apartment], [lamborghini, veneno], [studying, english], [english, lite...|
|[[normal, uni], [uni, student], [particular, clubgroup], [crush, onwe], [onwe, used], [awkwardthe...|
|[[techniques, however], [bother, practicing], [haphazard, placebo], [cur

Create network

In [11]:
edges_df = bigrams_df.select("id", F.explode(F.col("bigrams")).alias("bigram"))
edges_df = edges_df.select(
    F.col("bigram")[0].alias("src"),
    F.col("bigram")[1].alias("dst")
)

In [12]:
edges_df.show(20)

+------------+--------------------+
|         src|                 dst|
+------------+--------------------+
|frustratedly|               bored|
|       whole|                gone|
|        wind|        turnipspeech|
|    watching|            inuyasha|
|       found|             vaguely|
|     vaguely|        entertaining|
|       whole|selfimprovementthing|
|    starters|                 kit|
|      pretty|                long|
|        bold|               lines|
|        huge|                 fan|
|       never|               shied|
|       shied|                away|
|      always|               tried|
|         fit|             healthy|
|        mind|             however|
|      recent|               years|
|        huge|            problems|
|    problems|             keeping|
|  relatively|            speaking|
+------------+--------------------+
only showing top 20 rows



Create vertices

In [13]:
vertices_df = edges_df.select(F.col("src").alias("id")).union(edges_df.select(F.col("dst").alias("id"))).distinct()

In [14]:
vertices_df.show(10)

+------------+
|          id|
+------------+
|         art|
|conversation|
|       often|
|      taking|
|       watch|
|     explain|
|    romantic|
|    spending|
|        film|
| perspective|
+------------+
only showing top 10 rows



In [15]:
edges_df = edges_df.distinct()
edges_df.show(10)

+---------+-------------+
|      src|          dst|
+---------+-------------+
|   mental|        state|
|    wasnt|         home|
|  masters|       degree|
|     cold|relationships|
|  clearly|         knew|
|    dazed|       living|
|    needs|       taking|
|     head|       degree|
|   social|  environment|
|potential|      success|
+---------+-------------+
only showing top 10 rows



In [16]:
network = GraphFrame(vertices_df, edges_df)

/software/spark-3.3.2-el8-x86_64/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [17]:
network.inDegrees.orderBy(F.col("inDegree"), ascending=False).show()

/software/spark-3.3.2-el8-x86_64/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+----------+--------+
|        id|inDegree|
+----------+--------+
|      back|      38|
|     every|      32|
|       job|      28|
|     years|      26|
|    better|      26|
|      away|      22|
|everything|      21|
|      part|      20|
|   friends|      20|
|       try|      19|
|   instead|      18|
|      read|      16|
|      find|      16|
|   however|      16|
|    skills|      16|
|    please|      16|
|    social|      16|
|    either|      16|
|   therapy|      15|
|    others|      15|
+----------+--------+
only showing top 20 rows



In [18]:
results = network.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").orderBy(F.col("pagerank"), ascending=False).show(50)

+----------+------------------+
|        id|          pagerank|
+----------+------------------+
|    listen| 150.0070771995183|
|       bla|61.014216766194664|
|       woo| 37.65816250793299|
|      kata|26.371381215293447|
|      back|   22.057585711522|
|    slowly| 17.98323383450343|
|      away|16.562529466481717|
|      tell| 16.33501414825656|
|everything|14.483840559464932|
|     every|14.082245287232228|
|    better|12.965661898094059|
|       say| 12.63555834294155|
|   telling|12.332068662072956|
|  physical|12.306820079058182|
|   however|11.518418374300731|
|   whether|11.301621155705147|
|       try|11.171512419062235|
|      fate| 10.79617623684786|
|   friends|10.170346963061345|
|   stories| 9.874905394175311|
|  honestly| 9.690963302836591|
|  remember| 9.445595923992132|
|    rather| 9.347186365006074|
|     first| 8.998795010951117|
|    either| 8.974650744597717|
|     years| 8.958673160348038|
|     ampxb| 8.954279721399264|
|      long|  8.83670676901325|
|additio

## REMOVE SAMPLE